In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17213482769086738528, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9234857329
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13202867724487678116
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6692552704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7875403449622382567
 physical_device_desc: "device: 1, name: GeForce GTX 1070 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.10.0


In [5]:
batch_size = 8
img_rows, img_cols = 512, 512
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data/train/'
validation_dir = 'data/validation/'
test_dir = 'data/test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 9672 images belonging to 1 classes.
Found 9672 images belonging to 1 classes.
Found 3096 images belonging to 1 classes.
Found 3096 images belonging to 1 classes.
Found 150 images belonging to 1 classes.
Found 150 images belonging to 1 classes.


In [8]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [9]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 9672
nb_validation_samples: 3096
nb_test_samples: 150

predict_size_train: 1209
predict_size_validation: 387
predict_size_test: 19


In [10]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [11]:
from segmentation_models import PSPNet,Unet,Linknet, FPN
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

BACKBONE = 'inceptionv3'
preprocess_input = get_preprocessing(BACKBONE)

In [12]:
model = Linknet(BACKBONE, encoder_weights='imagenet')
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.8, beta_2=0.4)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    verbose = 2,                              
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
 - 569s - loss: -3.8778e-01 - dice_coef: 0.3878 - acc: 0.4574 - mean_squared_error: 0.2670 - Jaccard_index: 0.2430 - Specificity: 0.7769 - Sensitivity: 1.2473 - precision: 0.3304 - recall: 0.6127 - f1score: 0.4060 - val_loss: -3.9635e-01 - val_dice_coef: 0.3964 - val_acc: 0.3508 - val_mean_squared_error: 0.3156 - val_Jaccard_index: 0.2497 - val_Specificity: 0.7805 - val_Sensitivity: 1.3152 - val_precision: 0.3259 - val_recall: 0.8447 - val_f1score: 0.4628
Epoch 2/50
 - 553s - loss: -4.9908e-01 - dice_coef: 0.4991 - acc: 0.5944 - mean_squared_error: 0.2162 - Jaccard_index: 0.3375 - Specificity: 0.7923 - Sensitivity: 1.2100 - precision: 0.4558 - recall: 0.8707 - f1score: 0.5872 - val_loss: -5.8838e-01 - val_dice_coef: 0.5884 - val_acc: 0.7506 - val_mean_squared_error: 0.1633 - val_Jaccard_index: 0.4218 - val_Specificity: 0.7899 - val_Sensitivity: 1.1207 - val_precision: 0.5969 - val_recall: 0.8058 - val_f1score: 0.6818
Epoch 3/50
 - 552s - loss: -6.0868e-01 - dice_coef: 0.6087

Epoch 19/50
 - 552s - loss: -8.0463e-01 - dice_coef: 0.8046 - acc: 0.8654 - mean_squared_error: 0.1256 - Jaccard_index: 0.6809 - Specificity: 0.8085 - Sensitivity: 1.0470 - precision: 0.7835 - recall: 0.8297 - f1score: 0.8048 - val_loss: -6.9648e-01 - val_dice_coef: 0.6965 - val_acc: 0.7963 - val_mean_squared_error: 0.1950 - val_Jaccard_index: 0.5446 - val_Specificity: 0.7851 - val_Sensitivity: 1.0562 - val_precision: 0.6919 - val_recall: 0.7080 - val_f1score: 0.6965
Epoch 20/50
 - 552s - loss: -8.0714e-01 - dice_coef: 0.8071 - acc: 0.8674 - mean_squared_error: 0.1236 - Jaccard_index: 0.6841 - Specificity: 0.8088 - Sensitivity: 1.0461 - precision: 0.7864 - recall: 0.8318 - f1score: 0.8073 - val_loss: -6.8111e-01 - val_dice_coef: 0.6811 - val_acc: 0.7950 - val_mean_squared_error: 0.1962 - val_Jaccard_index: 0.5252 - val_Specificity: 0.7774 - val_Sensitivity: 1.0344 - val_precision: 0.7076 - val_recall: 0.6631 - val_f1score: 0.6811
Epoch 21/50
 - 552s - loss: -8.1031e-01 - dice_coef: 0.8

Epoch 37/50
 - 552s - loss: -8.3576e-01 - dice_coef: 0.8358 - acc: 0.8855 - mean_squared_error: 0.1055 - Jaccard_index: 0.7243 - Specificity: 0.8130 - Sensitivity: 1.0403 - precision: 0.8166 - recall: 0.8581 - f1score: 0.8358 - val_loss: -6.8396e-01 - val_dice_coef: 0.6840 - val_acc: 0.7941 - val_mean_squared_error: 0.1974 - val_Jaccard_index: 0.5299 - val_Specificity: 0.7807 - val_Sensitivity: 1.0453 - val_precision: 0.6957 - val_recall: 0.6793 - val_f1score: 0.6840
Epoch 38/50
 - 552s - loss: -8.3609e-01 - dice_coef: 0.8361 - acc: 0.8863 - mean_squared_error: 0.1048 - Jaccard_index: 0.7259 - Specificity: 0.8133 - Sensitivity: 1.0404 - precision: 0.8163 - recall: 0.8588 - f1score: 0.8362 - val_loss: -6.8878e-01 - val_dice_coef: 0.6888 - val_acc: 0.7957 - val_mean_squared_error: 0.1957 - val_Jaccard_index: 0.5347 - val_Specificity: 0.7804 - val_Sensitivity: 1.0426 - val_precision: 0.7029 - val_recall: 0.6808 - val_f1score: 0.6888
Epoch 39/50
 - 552s - loss: -8.3700e-01 - dice_coef: 0.8

In [ ]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

In [ ]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

In [ ]:
prediction.shape

In [ ]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

In [ ]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

In [ ]:
os.mkdir('02.LinkNet_ResNet50_results_dir')

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "02.LinkNet_ResNet50_results_dir"
img_rows, img_cols = 512, 512

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("02.LinkNet_ResNet50_results.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread(test_data_pred_dir+'/214_predict.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()